# Project: Summariazition

In [3]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)

True

## Basic Prompt

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

In [5]:
text= """
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

message = [
    SystemMessage(content='You are an expert copywriter with ecpertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0,
                 model_name='gpt-3.5-turbo')

In [6]:
llm.get_num_tokens(text)

231

In [7]:
summary_output = llm(message)

In [8]:
print(summary_output.content)

Mojo is a new programming language that combines the usability of Python with the performance of C. It aims to bridge the gap between research and production in the field of AI by offering portable code that is faster than C and seamlessly integrates with the Python ecosystem. Mojo was developed to simplify programming across the entire ML/AI infrastructure and to address the importance of host CPUs in AI acceleration.


## Prompt Templates

In [9]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [10]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [12]:
llm.get_num_tokens(prompt.format(text=text, language='Polish'))

251

In [13]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run({'text': text,
                     'language': 'Polish'})

In [14]:
summary

'Mojo to nowy język programowania, który łączy użyteczność Pythona z wydajnością C, odblokowując niezrównaną programowalność sprzętu AI i rozszerzalność modeli AI. Mojo łączy najlepsze cechy składni Pythona z programowaniem systemowym i metaprogramowaniem, umożliwiając pisanie przenośnego kodu, który jest szybszy niż C i bezproblemowo współpracuje z ekosystemem Pythona. Mojo powstał w celu ułatwienia programowania na całym stosie technologicznym w dziedzinie uczenia maszynowego i sztucznej inteligencji. Język ten umożliwia również wykorzystanie CPU jako akceleratora, który może obsługiwać operacje, których specjalizowane akceleratory nie są w stanie przetworzyć.'

## StuffDocumentsChain

In [15]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [17]:
with open ('../99-Data/sj.txt', encoding='utf-8') as f:
    text = f.read()
text

'I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.\n\nThe first story is about connecting the dots.\n\nI dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?\n\nIt started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting list, got a call in the middle of the night as

In [18]:
docs = [Document(page_content=text)]

In [19]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [27]:
template = '''Write a concise and short summary of the following text.
Text: {text}
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [28]:
chains = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chains.run(docs)

In [29]:
print(output_summary)

The speaker, who never graduated from college, shares three stories from his life. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from the company he co-founded, Apple, and how it allowed him to start over and eventually lead to the success of Pixar and his return to Apple. The third story is about facing death when he was diagnosed with cancer and how it made him realize the importance of living life to the fullest. The speaker encourages the audience to follow their passions, trust their intuition, and not waste time living someone else's life. He ends with a quote from The Whole Earth Catalog: "Stay Hungry. Stay Foolish."


## Summarazing Large Documents Using map_reduce

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [31]:
with open ('../99-Data/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [32]:
llm.get_num_tokens(text)

2653

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5_000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [38]:
len(chunks)

3

In [39]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.run(chunks)

In [40]:
print(output_summary)

Steve Jobs shares three stories from his life in his commencement speech. He talks about dropping out of college and how it led to the development of beautiful typography in the Macintosh computer. He also discusses love and loss, as well as his experience with a cancer diagnosis. Jobs emphasizes the importance of following curiosity, finding what you love, and living each day to the fullest. He encourages the audience to stay true to themselves and their intuition.


In [41]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [42]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

## Map_reduce with Custom Prompts

In [43]:
map_prompt = '''
Write a short a concise summary of the following:
Text: `{text}`
Concise Summary:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt   
)

In [44]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(
    template=combine_prompt,
    input_variables=['text']
)

In [45]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=True
)
output = summary_chain.run(chunks)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a short a concise summary of the following:
Text: `I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?

It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when

In [46]:
print(output)

Title: Steve Jobs' Reflections on Life, Love, and Death

Introduction:
In a speech given by Steve Jobs, the co-founder of Apple, he shares personal stories and reflections on his experiences of dropping out of college, love and loss, and facing death. Jobs emphasizes the importance of trusting in the future, finding what you love, and living life to the fullest.

Key Points:
- Dropping out of college led to unexpected opportunities, such as learning calligraphy, which influenced the design of the Macintosh computer.
- Getting fired from Apple allowed Jobs to start over and ultimately led to his success with other ventures.
- Jobs reflects on the importance of finding what you love and doing great work.
- The speaker shares their experience with a rare form of pancreatic cancer that was successfully treated with surgery.
- Jobs reflects on the inevitability of death and encourages the audience to live their own lives and follow their intuition.
- The speaker mentions the publication The

## Summarization using refine CombineDocumentChain

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
loader = UnstructuredPDFLoader('../99-Data/attention_is_all_you_need.pdf')
data = loader.load()

In [8]:
# print(data[0].page_content)

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10_000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [10]:
len(chunks)

4

In [11]:
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo')

In [13]:
def print_embedding_cost(text):
    import tiktoken
    enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in text])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.002:.6f}')

print_embedding_cost(chunks)

Total Tokens: 9650
Embedding Cost in USD: 0.019300


In [14]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=False
)
output_summary = chain.run(chunks)

In [15]:
print(output_summary)

The paper introduces the Transformer, a new network architecture based solely on attention mechanisms. The Transformer model achieves superior results in machine translation tasks and offers advantages such as parallelizability and reduced training time compared to existing models. The paper discusses the benefits of self-attention and describes the architecture of the Transformer model, which consists of stacked self-attention and fully connected layers in both the encoder and decoder. The attention mechanism used in the model is called Scaled Dot-Product Attention. Additionally, the paper explores the use of multi-head attention and positional encodings in the Transformer model. The authors compare self-attention layers to recurrent and convolutional layers in terms of computational complexity, parallelizability, and the ability to learn long-range dependencies. The paper also highlights the interpretability of self-attention and presents examples of attention distributions. The trai

  ## Refine Chain with Custom Prompts

In [17]:
prompt_template = '''
Write a cincise summary of the following extracting the key information:
Text: `{text}`
Concise Summary:'''
initial_prompt = PromptTemplate(template=prompt_template, 
                                input_variables=['text'])

refine_template = '''
Your job is to produce a final summary.
I have provided an existing summary up to a certain point: {existing_answer}.
Please refine the existing summary with some more context below.
--------
{text}
--------
Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHARSE
'''

refine_prompt = PromptTemplate(template=refine_template,
                               input_variables=['existing_answer', 'text']
                               )

In [19]:
chain = load_summarize_chain(llm=llm,
                             chain_type='refine',
                             question_prompt=initial_prompt,
                             refine_prompt=refine_prompt,
                             return_intermediate_steps=False
                             )
output_summary=chain.run(chunks)

In [20]:
print(output_summary)

Introduction:
The Transformer is a network architecture that relies solely on attention mechanisms, without the use of recurrent or convolutional neural networks. It has been shown to achieve superior quality in machine translation tasks, with faster training time and better parallelizability compared to existing models. The Transformer model also generalizes well to other tasks, such as English constituency parsing.

Key Points:
- The attention mechanism used in the Transformer is called Scaled Dot-Product Attention, which computes dot products of queries and keys and applies a softmax function to obtain weights.
- Multi-Head Attention is employed in the Transformer, allowing the model to jointly attend to information from different representation subspaces at different positions.
- The Transformer uses attention in three different ways: encoder-decoder attention, self-attention in the encoder, and self-attention in the decoder. These attention mechanisms enable the model to attend ov

## Summarizing Using LangChain Agents

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [22]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [23]:
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-3.5-turbo')

wikipedia = WikipediaAPIWrapper()

In [24]:
tools = [
    Tool(
        name='Wikipedia',
        func=wikipedia.run,
        description='Useful for when you need to get information from wikipedia about a topic'
    )
]

In [25]:
agent_executor = initialize_agent(tools, 
                                  llm,
                                  agent='zero-shot-react-description',
                                  verbose=True)

In [26]:
output = agent_executor.run('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should look up George Washington on Wikipedia to get a summary of his life and accomplishments.
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, politician and statesman who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in June 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted and ratified the Constitution of the United States and established the American federal government. Washington has thus been called the "Father of the Nation".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. He subsequently received mi

In [27]:
print(output)

George Washington was an American Founding Father, military officer, politician, and statesman who served as the first president of the United States from 1789 to 1797. He played a crucial role in the American Revolutionary War, led the drafting and ratification of the Constitution, and implemented a strong national government as president. He is considered one of the greatest presidents in American history.
